# Python Data Cleaning: NYC Airbnb Data from 2019
Source: [New York City Airbnb Open Data](https://www.kaggle.com/dgomonov/new-york-city-airbnb-open-data) 

*Please note the source data was modified to create situations for this tutorial.*

**Overview**
- [Set up and check data](#Set-up-and-check-data)
- [Dropping duplicate value rows](#Dropping-duplicate-value-rows)
- [Changing data types](#Changing-data-types)
- [Renaming columns](#Renaming-columns)
- [Dropping columns](#Dropping-columns)
- [Filling null values in categorical columns](#Filling-null-values-in-categorical-columns)
- [Filling null values in numerical columns](#Filling-null-values-in-numerical-columns)
- [Dropping rows with null values](#Dropping-rows-with-null-values)

### Set up and check data ###

In [1]:
# Load pandas and numpy library and stats from the scipy library
import pandas as pd
import numpy as np
from scipy import stats

In [2]:
# Load CSV of data as a pandas dataframe
raw_data = pd.read_csv('airbnb_nyc_2019.csv')

In [3]:
raw_data.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149.0,1,9,10/19/2018,0.21,6,365
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225.0,1,45,5/21/2019,0.38,2,355
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150.0,3,0,NaN,NaN,1,365
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89.0,1,270,7/5/2019,4.64,1,194
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80.0,10,9,11/19/2018,0.10,1,0


In [4]:
print(raw_data.index) # Python indexing starts at 0, unlike R, where indexing starts at 1.

RangeIndex(start=0, stop=48900, step=1)


In [5]:
raw_data.describe() # Summary statistics for each numeric column

,id,host_id,latitude,longitude,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
count,4.890000e+04,4.890000e+04,48900.000000,48900.000000,48888.000000,48900.000000,48900.000000,38846.000000,48900.000000,48900.000000
mean,1.901753e+07,6.761978e+07,40.728952,-73.952166,152.720074,7.030102,23.273804,1.373266,7.143374,112.780593
std,1.098340e+07,7.861036e+07,0.054535,0.046165,240.166390,20.509799,44.548793,1.680489,32.950889,131.620979
min,2.539000e+03,2.438000e+03,40.499790,-74.244420,0.000000,1.000000,0.000000,0.010000,1.000000,0.000000
25%,9.471971e+06,7.820478e+06,40.690098,-73.983072,69.000000,1.000000,1.000000,0.190000,1.000000,0.000000
50%,1.967743e+07,3.079257e+07,40.723075,-73.955680,106.000000,3.000000,5.000000,0.720000,1.000000,45.000000
75%,2.915244e+07,1.074344e+08,40.763120,-73.936270,175.000000,5.000000,24.000000,2.020000,2.000000,227.000000
max,3.648724e+07,2.743213e+08,40.913060,-73.712990,10000.000000,1250.000000,629.000000,58.500000,327.000000,365.000000


In [6]:
raw_data.info() # Given index #, column name, counts, and data types for entire dataframe

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48900 entries, 0 to 48899
Data columns (total 16 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              48900 non-null  int64  
 1   name                            48884 non-null  object 
 2   host_id                         48900 non-null  int64  
 3   host_name                       48879 non-null  object 
 4   neighbourhood_group             48900 non-null  object 
 5   neighbourhood                   48900 non-null  object 
 6   latitude                        48900 non-null  float64
 7   longitude                       48900 non-null  float64
 8   room_type                       48900 non-null  object 
 9   price                           48888 non-null  float64
 10  minimum_nights                  48900 non-null  int64  
 11  number_of_reviews               48900 non-null  int64  
 12  last_review                     

In [7]:
data = raw_data.copy() # Create a copy of the original dataframe to work with

### Dropping duplicate value rows ###
Drop any rows that have a duplicate ID, assuming this should be unique to each listing record.

In [8]:
data = data.drop_duplicates(subset=['id'])

### Changing data types ### 
Change the last_review column to datetime objects.

In [9]:
data['last_review'] =  pd.to_datetime(data['last_review'], format='%m/%d/%Y')

In [10]:
data['last_review'].head()

0   2018-10-19
1   2019-05-21
2          NaT
3   2019-07-05
4   2018-11-19
Name: last_review, dtype: datetime64[ns]

### Renaming columns ### 
Change the neighbourhood columns to reflect typical spelling of the word in American English by dropping the 'u'.

In [11]:
data = data.rename(columns = {'neighbourhood_group' : 'neighborhood_group', 'neighbourhood' : 'neighborhood'})

In [12]:
data.iloc[:, 4:6].head()

,neighborhood_group,neighborhood
0,Brooklyn,Kensington
1,Manhattan,Midtown
2,Manhattan,Harlem
3,Brooklyn,Clinton Hill
4,Manhattan,East Harlem


### Dropping columns ###
Create a new column and then remove it.

In [13]:
data['test_col'] = 5

In [14]:
data['test_col'].head()

0    5
1    5
2    5
3    5
4    5
Name: test_col, dtype: int64

In [15]:
data.drop('test_col', axis=1, inplace=True)

In [16]:
data.head()

,id,name,host_id,host_name,neighborhood_group,neighborhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149.0,1,9,2018-10-19,0.21,6,365
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225.0,1,45,2019-05-21,0.38,2,355
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150.0,3,0,NaT,NaN,1,365
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89.0,1,270,2019-07-05,4.64,1,194
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80.0,10,9,2018-11-19,0.10,1,0


### Filling null values in categorical columns ### 
Fill blanks in the 'name' and 'host_name' columns to signify the data was not provided.

In [17]:
data[data.isnull().any(axis=1)]

,id,name,host_id,host_name,neighborhood_group,neighborhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150.0,3,0,NaT,NaN,1,365
20,7750,Huge 2 BR Upper East Cental Park,17985,Sing,Manhattan,East Harlem,40.79685,-73.94872,Entire home/apt,190.0,7,0,NaT,NaN,2,249
27,8700,Magnifique Suite au N de Manhattan - vue Cloitres,26394,Claude & Sophie,Manhattan,Inwood,40.86754,-73.92639,Private room,80.0,4,0,NaT,NaN,1,0
37,11452,Clean and Quiet in Brooklyn,7355,Vt,Brooklyn,Bedford-Stuyvesant,40.68876,-73.94312,Private room,35.0,60,0,NaT,NaN,1,365
39,11943,Country space in the city,45445,Harriet,Brooklyn,Flatbush,40.63702,-73.96327,Private room,150.0,1,0,NaT,NaN,1,365
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48895,36484665,Charming one bedroom - newly renovated rowhouse,8232441,Sabrina,Brooklyn,Bedford-Stuyvesant,40.67853,-73.94995,Private room,70.0,2,0,NaT,NaN,2,9
48896,36485057,Affordable room in Bushwick/East Williamsburg,6570630,Marisol,Brooklyn,Bushwick,40.70184,-73.93317,Private room,40.0,4,0,NaT,NaN,2,36
48897,36485431,Sunny Studio at Historical Neighborhood,23492952,Ilgar & Aysel,Manhattan,Harlem,40.81475,-73.94867,Entire home/apt,115.0,10,0,NaT,NaN,1,27
48898,36485609,43rd St. Time Square-cozy single bed,30985759,Taz,Manhattan,Hell's Kitchen,40.75751,-73.99112,Shared room,55.0,1,0,NaT,NaN,6,2


In [18]:
data.sort_values(by = 'host_name')

,id,name,host_id,host_name,neighborhood_group,neighborhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
38879,30425770,Queens Corner Lot,128580688,#NAME?,Queens,Queens Village,40.70524,-73.73349,Entire home/apt,150.0,1,51,2019-07-07,7.43,1,248
11138,8622301,Astoria Full Basement Apartment,45354224,'Cil,Queens,Astoria,40.76914,-73.92118,Private room,120.0,2,240,2019-07-01,5.22,1,298
10316,7899658,MANHATTAN LIVING!,40605120,(Ari) HENRY LEE,Manhattan,East Harlem,40.78938,-73.94679,Entire home/apt,140.0,7,3,2017-12-08,0.06,1,56
5331,3848324,West Village apt - 2 Bedroom,11495251,(Email hidden by Airbnb),Manhattan,West Village,40.73123,-74.00428,Entire home/apt,200.0,5,1,2014-08-30,0.02,1,0
35173,27903031,Prewar Gem in Middle of Everything!,7580102,(Email hidden by Airbnb),Manhattan,Midtown,40.76399,-73.98077,Private room,139.0,3,3,2018-11-17,0.31,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28044,21901156,Best Deal! Lovely place in Manhattan! Time Squ...,159156636,NaN,Manhattan,Hell's Kitchen,40.75656,-73.99063,Private room,120.0,1,109,2019-01-01,5.97,3,0
28276,21990808,***SLEEPING IN THE GOLDEN PLACE***,159156636,NaN,Manhattan,Hell's Kitchen,40.75835,-73.99065,Private room,120.0,1,88,2018-12-14,4.93,3,0
30572,23660605,Private Bedroom w/ Balcony view of Central Park,177146433,NaN,Manhattan,East Harlem,40.79766,-73.94824,Private room,139.0,1,6,2018-08-12,0.46,1,0
32195,25180572,Beautiful Cozy Studio Apt. UWS,119609345,NaN,Manhattan,Upper West Side,40.77701,-73.97667,Entire home/apt,180.0,4,0,NaT,NaN,1,0


In [19]:
data['host_name'].nunique() # Get count of unique host names. Because there are many, we will focus on replcaing NaNs

11452

In [20]:
data['name'] = data['name'].fillna('Not Provided')

In [21]:
data['host_name'] = data['host_name'].fillna('Not Provided')

In [22]:
# Other examples seen above, replacing specific strings '(Email hidden by Airbnb)' and '#NAME?'
data = data.replace(to_replace = ('(Email hidden by Airbnb)', '#NAME?'), value = 'Not Provided')

In [23]:
data.sort_values(by = 'host_name')

,id,name,host_id,host_name,neighborhood_group,neighborhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
11138,8622301,Astoria Full Basement Apartment,45354224,'Cil,Queens,Astoria,40.76914,-73.92118,Private room,120.0,2,240,2019-07-01,5.22,1,298
10316,7899658,MANHATTAN LIVING!,40605120,(Ari) HENRY LEE,Manhattan,East Harlem,40.78938,-73.94679,Entire home/apt,140.0,7,3,2017-12-08,0.06,1,56
19545,15628789,Newly Renovate 2 Bedroom Apartment in Central ...,100757200,(Mary) Haiy,Brooklyn,Bay Ridge,40.63461,-74.02451,Entire home/apt,126.0,1,188,2019-06-24,5.81,1,295
39436,30735480,Neve recording studio,229693987,123,Manhattan,Lower East Side,40.71456,-73.98717,Entire home/apt,600.0,1,0,NaT,NaN,1,365
33966,26895166,2018Serenity,202318295,2018Serenity,Manhattan,East Harlem,40.79424,-73.94290,Private room,50.0,5,4,2019-06-22,0.39,1,32
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41885,32554746,"Clean, Safe, east village room!",112439306,빈나,Manhattan,Stuyvesant Town,40.73180,-73.97999,Private room,125.0,4,1,2019-03-06,0.24,1,0
47266,35662019,Cheap price near Timesquare,79269209,소정,Manhattan,Hell's Kitchen,40.75625,-73.99357,Private room,63.0,31,0,NaT,NaN,1,250
33879,26834353,Cozy Brooklyn Room - Next to Pratt Institute,133425456,소정,Brooklyn,Bedford-Stuyvesant,40.69056,-73.95980,Private room,40.0,5,1,2018-08-18,0.09,1,0
29172,22392030,one bedroom apt for long term stay up to 2 months,151728547,진,Manhattan,Upper West Side,40.77506,-73.98162,Entire home/apt,70.0,15,0,NaT,NaN,1,0


### Filling null values in numerical columns ### 
Use mean, median, and mode to get possible values to use instead of NaN numeric values.

In [24]:
null_price_ind = data[data['price'].isnull()].index.tolist()
impute_price = data.copy()
missing_prices = impute_price.index.isin(null_price_ind)
impute_price = impute_price[~ missing_prices]

In [25]:
mean = int(round(np.mean(impute_price['price'])))
median = np.median(impute_price['price'])
mode = stats.mode(impute_price['price'])

print("\tmean: {}".format(mean))
print("\tmedian: {}".format(median))
print("\tmode: {}".format(mode[0]))

	mean: 153
	median: 106.0
	mode: [100.]


In [26]:
# Using the median to fill in null prices
data['price'] = data['price'].fillna(median)

In [27]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 48895 entries, 0 to 48899
Data columns (total 16 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   id                              48895 non-null  int64         
 1   name                            48895 non-null  object        
 2   host_id                         48895 non-null  int64         
 3   host_name                       48895 non-null  object        
 4   neighborhood_group              48895 non-null  object        
 5   neighborhood                    48895 non-null  object        
 6   latitude                        48895 non-null  float64       
 7   longitude                       48895 non-null  float64       
 8   room_type                       48895 non-null  object        
 9   price                           48895 non-null  float64       
 10  minimum_nights                  48895 non-null  int64         
 11  nu

### Dropping rows with null values ### 
Drop rows of a dataframe if they have a null value in a specific column.

In [28]:
drop_null_data = raw_data.copy()
drop_null_data = drop_null_data.dropna(subset=['reviews_per_month'])

In [29]:
drop_null_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 38846 entries, 0 to 48857
Data columns (total 16 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              38846 non-null  int64  
 1   name                            38840 non-null  object 
 2   host_id                         38846 non-null  int64  
 3   host_name                       38830 non-null  object 
 4   neighbourhood_group             38846 non-null  object 
 5   neighbourhood                   38846 non-null  object 
 6   latitude                        38846 non-null  float64
 7   longitude                       38846 non-null  float64
 8   room_type                       38846 non-null  object 
 9   price                           38837 non-null  float64
 10  minimum_nights                  38846 non-null  int64  
 11  number_of_reviews               38846 non-null  int64  
 12  last_review                     